In [ ]:
import pandas as pd
import numpy as np
import itertools
import re

from ast import literal_eval
from collections import Counter
from matplotlib import pyplot as plt
from sklearn import metrics

from nltk.stem import PorterStemmer
from sklearn.calibration import CalibratedClassifierCV
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

%matplotlib inline

In [ ]:
df = pd.read_csv('../data/lobsters_full_2017_cleaned.csv')

In [ ]:
df.head()

In [ ]:
df = df.set_index('Unnamed: 0')

### Evaluating Tags to Use

In [ ]:
df.tags.values

In [ ]:
literal_eval(df.tags.values[0])

In [ ]:
tag_list = [literal_eval(tag) for tag in df.tags.values]

In [ ]:
counter = Counter(itertools.chain(*tag_list))
counter.most_common(10)

In [ ]:
%matplotlib inline
n = 10
most_common = counter.most_common(n)
plt.barh(range(n), 
         [mc[1] for mc in most_common], 
        tick_label=[mc[0] for mc in most_common])

In [ ]:
my_categories = ['security', 'web', 'hardware', 'culture']

In [ ]:
itertools.combinations?

In [ ]:
for first_cat, second_cat in itertools.combinations(my_categories, 2):
    overlap = df[(df[first_cat] == 1) & 
                 (df[second_cat] == 1)].shape[0]
    print(first_cat, second_cat, overlap)

In [ ]:
sum(df['security'])

In [ ]:
sum(df['security'].fillna(0))

In [ ]:
sum(df['web'].fillna(0))

In [ ]:
sum(df['hardware'].fillna(0))

In [ ]:
# overlap between security and web (for web group)
252 / 2263

In [ ]:
# ovelap between security and hardware (for hardware group)
174 / 1537

### Your Turn

- evaluate a few other categories that might be interesting to you
- choose a final `my_categories` list 

### Preprocessing Text

In [ ]:
df[my_categories].sum(axis=1) == 1

In [ ]:
columns = my_categories + ['title', 'description']
tag_data = df[df[my_categories].sum(axis=1) == 1][columns]

In [ ]:
tag_data.head()

In [ ]:
all_tags = counter.keys()
stemmer = PorterStemmer()

def clean_text(sentence):
    words = re.findall("\w+", sentence.lower())
    return [word for word in words if
            word not in ENGLISH_STOP_WORDS
            and (len(word) > 1 or word in all_tags)]


def remove_urls(text):
    return re.sub('http\S+', '', text)

In [ ]:
def tokenize(text):
    return [stemmer.stem(w) for w in 
            clean_text(remove_urls(text.lower()))]

In [ ]:
test_text = ' '.join([
    tag_data.iloc[0]['title'], 
    tag_data.iloc[0]['description']])
test_text

In [ ]:
tag_data = tag_data.fillna('')

In [ ]:
tokenize(test_text)

In [ ]:
tag_data['full_text'] = tag_data['title'] + ' ' + tag_data['description']

In [ ]:
tag_data.head()

In [ ]:
tokenize(tag_data.iloc[3]['full_text'])

### Train Test Split

In [ ]:
labels = [(k, v) for k, v in enumerate(my_categories)]

In [ ]:
labels

In [ ]:
def label_row(row, labels=labels):
    for label, tag_name in labels:
        if row[tag_name] == 1:
            return label

In [ ]:
tag_data['tag_class'] = tag_data.apply(label_row, axis=1)

In [ ]:
tag_data.head()

In [ ]:
y = tag_data.tag_class
X = tag_data['full_text'].as_matrix()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

In [ ]:
print("training data: ", X_train.shape[0])
print("testing data: ", X_test.shape[0])


### Create Vectorizers

In [ ]:
count_vectorizer = CountVectorizer(tokenizer=tokenize,
                                  min_df=5, max_df=.9)

In [ ]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=tokenize,
                                  min_df=5, max_df=.9)

### Create a Bayesian model

In [ ]:
bayes_model = Pipeline([('vectorizer', count_vectorizer), 
                        ('clf', MultinomialNB())])

bayes_model.fit(X_train, y_train)

### Test Performance

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    See full source and example: 
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
predicted = bayes_model.predict(X_test)

In [ ]:
np.mean(predicted == y_test)

In [ ]:
cm = metrics.confusion_matrix(y_test, predicted)
plot_confusion_matrix(cm, my_categories)

In [ ]:
bayes_model.predict_proba(
    ["Brand new GPUs from Nvidea."])

### Try a SVM

In [ ]:
svm_model = Pipeline([('vectorizer', tfidf_vectorizer), 
                      ('clf', LinearSVC())])

svm_model.fit(X_train, y_train)

In [ ]:
svm_predicted = svm_model.predict(X_test)

In [ ]:
np.mean(svm_predicted == y_test)

In [ ]:
svm_cm = metrics.confusion_matrix(y_test, svm_predicted)
plot_confusion_matrix(svm_cm, my_categories)

### Your Turn

- Try at least one more model combination (i.e. count with LinearSVC, tfidf with MultinomialNB, or try the PassiveAggressiveClassifier which is imported as well!)
- Test the accuracy using np.mean
- Plot the confusion matrix
- Share what you found in our discussion and on Slack!
- Bonus: Read https://machinelearningmastery.com/tactics-to-combat-imbalanced-classes-in-your-machine-learning-dataset/ and try implementing a suggestion to better train our models!

## Parameter tuning

In [ ]:
svm_model.steps

In [ ]:
svm_model.steps[1][1]

In [ ]:
LinearSVC?

In [ ]:
train_vectors = tfidf_vectorizer.transform(X_train)

In [ ]:
np.logspace?

In [ ]:
Cs = np.logspace(-6, 6, 20)
Cs

In [ ]:
param_val = GridSearchCV(svm_model.steps[1][1], 
                   param_grid={
                       'C': Cs,
                       'loss': ['hinge', 'squared_hinge']
                   },
                   n_jobs=4)
param_val.fit(train_vectors, y_train)        

In [ ]:
param_val.best_score_

In [ ]:
param_val.best_estimator_.C

In [ ]:
param_val.best_estimator_.loss

In [ ]:
svm_model.steps[1][1].get_params()

In [ ]:
svm_model.steps[1][1].set_params(
    C=param_val.best_estimator_.C)

In [ ]:
svm_model.fit(X_train, y_train)

In [ ]:
predicted = svm_model.predict(X_test)

In [ ]:
np.mean(predicted == y_test)

### Your Turn

- Try GridSearchCV with the model you built. Did you find a new alpha or constant to use?
- If so, update your model parameters and train and test again
- Did you see any improvement? Why or why not?

## Save your model and training data

In [ ]:
from sklearn.externals import joblib
joblib.dump(bayes_model, 
            '../data/lobsters_tag_classification_bayes.pkl')
joblib.dump(svm_model, 
            '../data/lobsters_tag_classification_svm.pkl')

In [ ]:
tag_data.to_csv('../data/lobsters_tag_training_data.csv')

In [ ]:
import json
json.dump(dict((str(k), v)
          for k, v in enumerate(my_categories)),
          open('../data/lobsters_tag_classes.json', 'w'))
json.dump(list(all_tags), 
          open('../data/lobsters_tag_list.json', 'w'))

In [ ]:
!cat ../data/lobsters_tag_classes.json

### SVM with predict_proba

In [ ]:
svm = LinearSVC()
clf = CalibratedClassifierCV(svm) 

svm_model = Pipeline([('vectorizer', tfidf_vectorizer), 
                      ('clf', clf)])

svm_model.fit(X_train, y_train)
joblib.dump(svm_model, 
            '../data/lobsters_tag_classification_svm_proba.pkl')